In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Let's begin by loading our dataset and see how it looks

In [ ]:
train_data = pd.read_csv('/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv')
train_data

In [ ]:
train_data.describe()

In [ ]:
train_data.info()

In [ ]:
from pandas_profiling import ProfileReport


In [ ]:
report = ProfileReport(train_data)


# Plotting our data 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
targets = train_data.target.value_counts()

plt.figure(figsize=[3,4])
sns.barplot(x= targets.index, y= targets.values)
plt.show()

In this plot we can see that our data is unbalanced, the number of people willing to leave thier jobs is way smaller than the number of people not looking to change jobs.

In [ ]:
features_to_plot = ["city_development_index", "training_hours"]
plt.figure(figsize = [15, 17])
n = 1
for f in features_to_plot:
    plt.subplot(4,2,n)
    sns.histplot(x= f, hue = "target", alpha = 1, multiple = 'stack', data = train_data, bins= 10) 
    plt.title("ditsebution of target  by {} ".format(f))
    plt.legend(['Not Looking for a job change', ' Looking for a job change'])
    sns.despine()
    n+=1
plt.tight_layout()
plt.show()

Here we can see that that people living in a city with a development index around 0.65 and 0.9 are more willing to find a new job. (fig 1)

we can see also that people with less training hours are more willing to change thier jobs. (fig 2)

**plotting the number of people looking for a job change against evry feature**

In [ ]:
to_plot = ['relevent_experience', 'enrolled_university', 'education_level','major_discipline', 'company_size', 'company_type', 'last_new_job']

plt.figure(figsize = [15, 17])
n = 1
for f in to_plot:
    plt.subplot(4,2,n)
    sns.countplot(x= train_data[f][train_data.target == 1], alpha = 1) 
    plt.title("countplot of people looking for a job change by {}".format(f))
    sns.despine()
    n+=1
plt.tight_layout()
plt.show()

# Handling missing values

In [ ]:
print(train_data.isnull().values.any())

In [ ]:
print(train_data.isnull().sum())

In [ ]:
def cleanNaN(dfA):
  for col in dfA.columns.to_list():
    if str(dfA[col].dtype) == 'object':
        dfA[col] = dfA[col].fillna('unknown')
    else:
        dfA[col] = dfA[col].fillna(dfA.mean())
  return dfA

In [ ]:
train_data = cleanNaN(train_data)
print(train_data.isnull().sum())

# balancig our dataset

In [ ]:
print(train_data.target.value_counts())

In [ ]:
stayed = train_data[train_data.target == 1]
left   = train_data[train_data.target == 0].sample(4800)
train_data = pd.concat([stayed, left], axis= 0)
print(train_data.shape)

In [ ]:
print(train_data.target.value_counts())


# Preparing our data for modeling

**handling categorical features**

In [ ]:
from sklearn import preprocessing

In [ ]:
categorical = ['city','relevent_experience','gender' , 'enrolled_university', 'education_level','major_discipline', 'last_new_job', 'experience', 'company_size', 'company_type']
encoder = preprocessing.LabelEncoder()
for f in categorical:
    train_data[f] = encoder.fit_transform(train_data[f])
train_data.info()

In [ ]:
print(train_data.isnull().sum())

**Spliting our dataset**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train = train_data.drop(columns=['enrollee_id','target'])
target = train_data.target

In [ ]:
scaler = preprocessing.MinMaxScaler()

In [ ]:
train = scaler.fit_transform(train.to_numpy())

In [ ]:
X_train,X_test, Y_train, Y_test = train_test_split(train, target, test_size=0.1, random_state=9, shuffle = True )

# Modle building

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
regressor = LogisticRegression()

In [ ]:
regressor.fit(X_train, Y_train)
predictions= regressor.predict(X_test)


In [ ]:
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,classification_report
print(f"Roc-Auc score: {roc_auc_score(Y_test,predictions)},f1_score: {f1_score(Y_test,predictions)},Accuracy: {accuracy_score(Y_test,predictions)}")
print(classification_report(Y_test,predictions))

In [ ]:
from sklearn.model_selection import cross_val_score,StratifiedKFold        # lets validate our val_accuracy
skfold = StratifiedKFold(n_splits=3, random_state=42,shuffle=True)
scores=cross_val_score(regressor,X_test,Y_test,cv=skfold)
print("best score:{:.3f}".format(np.mean(scores)))    

# Final thoughts

- People with relevant experience are more willing to take the new job.
- People who never enrolled in a college class are more willing to take the new job.
- By far Graduates, STEM majors and people working in a Pvt ltd are more willing to find a new job.


Thank you for reading this notebook, please upvote it and leave a comment.